In [1]:
import pandas as pd
jurnal = pd.read_csv(r"C:\Users\acer\anaconda3\envs\ki\datajurnal2.csv",sep=";")

In [2]:
jurnal.head()
#jurnal["similarity"]

,journalId,title,abstract,keywords
0,1,Mesenchymal Transformation: The Rosetta Stone ...,"Despite decades of research, glioblastoma (GB...",clinical outcome|glioblastoma (GBM)|mesenchyma...
1,2,"DGAT1 protects tumor from lipotoxicity, emergi...","We recently demonstrated that glioblastoma, th...",DGAT1|glioblastoma|lipotoxicity|oxidative stre...
2,3,Anti-PD-1 Immunotherapy in Preclinical GL261 G...,Glioblastomas (GBs) are malignant brain tumou...,TMZ|checkpoint inhibitors|glioblastoma|long-te...
3,4,Recent Updates on the Relationship between Can...,Autoimmune pancreatitis (AIP) is now considere...,IgG4-related disease|autoimmune pancreatitis|c...
4,5,Cancer and cure: A critical analysis,Cancer is one of the most dreaded diseases of ...,cancer|cure.


In [3]:
import pickle

from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [4]:
import re

def clean_abstract(abstract):
  re.sub("[a-zA-Z ]", "", abstract)
  return abstract

In [5]:
jurnal["clean_abstract"] = jurnal["abstract"].apply(clean_abstract)

In [6]:
jurnal['word_count'] = jurnal['abstract'].apply(lambda x: len(x.split()))
#jurnal.groupby('keywords')['word_count'].mean()
jurnal.groupby('abstract')['word_count'].mean()
jurnal


,journalId,title,abstract,keywords,clean_abstract,word_count
0,1,Mesenchymal Transformation: The Rosetta Stone ...,"Despite decades of research, glioblastoma (GB...",clinical outcome|glioblastoma (GBM)|mesenchyma...,"Despite decades of research, glioblastoma (GB...",676
1,2,"DGAT1 protects tumor from lipotoxicity, emergi...","We recently demonstrated that glioblastoma, th...",DGAT1|glioblastoma|lipotoxicity|oxidative stre...,"We recently demonstrated that glioblastoma, th...",920
2,3,Anti-PD-1 Immunotherapy in Preclinical GL261 G...,Glioblastomas (GBs) are malignant brain tumou...,TMZ|checkpoint inhibitors|glioblastoma|long-te...,Glioblastomas (GBs) are malignant brain tumou...,1019
3,4,Recent Updates on the Relationship between Can...,Autoimmune pancreatitis (AIP) is now considere...,IgG4-related disease|autoimmune pancreatitis|c...,Autoimmune pancreatitis (AIP) is now considere...,560
4,5,Cancer and cure: A critical analysis,Cancer is one of the most dreaded diseases of ...,cancer|cure.,Cancer is one of the most dreaded diseases of ...,452
...,...,...,...,...,...,...
85,86,Exact Controllability and Asymptotic Analysis ...,the authors consider the exact controllability...,NaN,the authors consider the exact controllability...,75
86,87,Existence and Asymptotic Behavior of Radially ...,This paper is concerned with a class of semili...,NaN,This paper is concerned with a class of semili...,127
87,88,Global Bifurcation of a Perturbed Double-Homoc...,This paper deals with a kind of fourth degree ...,NaN,This paper deals with a kind of fourth degree ...,34
88,89,Global Existence of Classical Solutions for So...,"In this paper, we prove local and global exist...",NaN,"In this paper, we prove local and global exist...",36


**Vectorizer**

In [7]:
from numpy.lib.function_base import vectorize
from sklearn.feature_extraction.text import TfidfVectorizer #mengubah kata menjadi angka

vectorizer = TfidfVectorizer(ngram_range=(1, 2))

tfidf = vectorizer.fit_transform(jurnal["clean_abstract"])
print(tfidf)

  (0, 13572)	0.02308226911540255
  (0, 12282)	0.02308226911540255
  (0, 14081)	0.02308226911540255
  (0, 1567)	0.02308226911540255
  (0, 12937)	0.02113963449775765
  (0, 12125)	0.02113963449775765
  (0, 6113)	0.02308226911540255
  (0, 14139)	0.02308226911540255
  (0, 5653)	0.02308226911540255
  (0, 12286)	0.02308226911540255
  (0, 6628)	0.02113963449775765
  (0, 2791)	0.02113963449775765
  (0, 925)	0.02113963449775765
  (0, 474)	0.02308226911540255
  (0, 10872)	0.02308226911540255
  (0, 4210)	0.02308226911540255
  (0, 14964)	0.01976131330156837
  (0, 14825)	0.02113963449775765
  (0, 11312)	0.01587604406627856
  (0, 4231)	0.02308226911540255
  (0, 541)	0.02308226911540255
  (0, 13616)	0.02308226911540255
  (0, 512)	0.02308226911540255
  (0, 13502)	0.02308226911540255
  (0, 4484)	0.02308226911540255
  :	:
  (89, 13858)	0.04383193391821544
  (89, 7306)	0.037147320100416145
  (89, 9240)	0.07317048936136361
  (89, 6637)	0.05777819959947229
  (89, 14771)	0.02736217360553039
  (89, 1868)	0.05

**1. similarity berdasarkan title**

In [8]:
from sklearn.metrics.pairwise import cosine_similarity # menggunakan pairwise untuk menghitung kedekatan antar kata
import numpy as np

#def search(title)
def search(abstract): # Membuat fungsi search
    print(abstract)
    abstract = clean_abstract(abstract) # Memanggil fungsi clean_title
    query_vec = vectorizer.transform([abstract]) # Mengubah isi dataframe title menjadi vektor
    similarity = cosine_similarity(query_vec, tfidf).flatten() # Perhitungan cosinus antara pencarian kata dengan dataset
    indices = np.argpartition(similarity, -5)[-5:] # Mendapatkan 10 data terdekat dari hasil pencarian
    results = jurnal.iloc[indices].iloc[::-1] # Menunjukan lokasi data bedasarkan indices

    print(similarity[indices])
    return results

search("Cancer comprises a bewildering assortment of diseases that kill 7.5 million people each year. Poor understanding of cancer's diversity currently thwarts our goal of a cure for every patient, but recent integration of genomic and stem cell technologies promises a route through this impasse.")

Cancer comprises a bewildering assortment of diseases that kill 7.5 million people each year. Poor understanding of cancer's diversity currently thwarts our goal of a cure for every patient, but recent integration of genomic and stem cell technologies promises a route through this impasse.
[0.07018458 0.0799747  0.08040725 0.36633039 0.10299155]


,journalId,title,abstract,keywords,clean_abstract,word_count
4,5,Cancer and cure: A critical analysis,Cancer is one of the most dreaded diseases of ...,cancer|cure.,Cancer is one of the most dreaded diseases of ...,452
13,14,Mapping cancer origins,Cancer comprises a bewildering assortment of d...,cancer,Cancer comprises a bewildering assortment of d...,409
18,19,Pathophysiology of lung cancer,Cigarette smoking is responsible for approxima...,cancer|lung cancer.,Cigarette smoking is responsible for approxima...,198
19,20,The Role of Telomerase in Breast Cancer's Resp...,"Currently, breast cancer appears to be the mos...",breast cancer|resistance|telomerase|therapy|ca...,"Currently, breast cancer appears to be the mos...",169
12,13,Early epigenetic cancer decisions,Abstract A cancer dogma states that inactivati...,cancer|tumor.,Abstract A cancer dogma states that inactivati...,229


In [9]:
# Membuat search bar interaktif
import ipywidgets as widgets
from IPython.display import display

jurnal_input = widgets.Text( # Variabel input search bar
    value='Cancer and cure: A critical analysis',
    description='abstract : ',
    disabled=False
)
jurnal_list = widgets.Output() # Menampilkan search bar

def on_type(data): # Mencari data pada setiap ketikan input
    with jurnal_list:
            jurnal_list.clear_output()
            title = data["new"]
            if len(abstract) > 5: # Membatasi minimal 5 karakter input untuk search
                display(search(abstract)) # Menampilkan hasil search kepada output widget

jurnal_input.observe(on_type, names='value') # Memanggil fungsi on_type ketika terjadi perubahan pada input bar

display(jurnal_input, jurnal_list) # Menampilkan kedua widget yang dibuat

Text(value='Cancer and cure: A critical analysis', description='abstract : ')

Output()

**2. Similarity berdasarkan Abstract**

In [19]:
from sklearn.metrics.pairwise import cosine_similarity # menggunakan pairwise untuk menghitung kedekatan antar kata
import numpy as np

#def search(abstract)
def search(abstract): # Membuat fungsi search
    print(abstract)
    #abstract =
    abstract = clean_abstract(abstract) # Memanggil fungsi clean_abstract
    query_vec = vectorizer.transform([abstract]) # Mengubah isi dataframe abstract menjadi vektor
    similarity = cosine_similarity(query_vec, tfidf).flatten() # Perhitungan cosinus antara pencarian kata dengan dataset
    indices = np.argpartition(similarity, -5)[-5:] # Menampilkan data terdekat dari hasil pencarian, mengurutkan nilai besar ke paling depan
    results = jurnal.iloc[indices].iloc[::-1] # Menunjukan lokasi data bedasarkan indices

    #jurnal['similarity'] = jurnal['abstract'].apply(similarity)
    #jurnal.groupby('clean_abstract')['similarity'].mean()
    #rec_percentages.head(10).merge(jurnal, left_index=True, right_on="title")
    #print(similarity)

    print(similarity[indices])
    return results

search("Breast cancer is the leading cause of cancer in women and the second leading cause of cancer-related death. There are many subtypes of breast cancer, which can be identified through the process of molecular and genetic profiling. While the current standard of care utilizes tumor tissue biopsy to subclassify breast cancer, plasma tumor DNA (ptDNA) can be detected through droplet digital PCR (ddPCR) of plasma obtained from a simple blood draw. Tissue biopsy is not only more invasive but because tumors exhibit heterogeneity it can be less accurate. Blood collects DNA shed from normal and cancerous cells alike, thus ddPCR of plasma offers a broader picture of a cancer's genetic makeup. This chapter summarizes how patients with breast cancer can be screened for specific cancerous mutations in both tissue and plasma through the use of ddPCR. Every year, over 230,000 women are diagnosed with breast cancer, and nearly 40,000 die from the disease. It is the most commonly diagnosed cancer in women and the second leading cause of cancerrelated death [1, 2]. Typically, breast cancer is detected via mammography and confirmed through tissue sampling. From here, patients undergo specific therapies targeted toward their class of breast cancer, which is defined principally by the presence or absence of estrogen receptor (ER), progesterone receptor (PR), and human epidermal growth factor receptor 2 (HER-2). Researchers have shown that within these subgroups, there are specific, idiosyncratic mutations, which can be useful for assessing burden of disease, predicting response to therapy, and monitoring the latter in a rapid and minimally invasive approach. These include mutations in PIK3CA, ESR1, TP53, MAP3K1, GATA3, RPTOR, ERBB2, and ERBB3 [3â€“6]. Current studies of receptor mutations, however, face a significant barrier: they rely on single site biopsy of tumor tissue. This method is problematic because tumors have been shown to exhibit heterogeneity, so sampling a single piece of tissue may miss sites of the specific mutations listed above. This same heterogeneity can exist within metastatic tumors, and furthermore, there has been documented heterogeneity between metastases at different sites of the body, even if the metastases originated from the same primary tumor [7, 8]. In other words, tissue biopsy limits clinicians from seeing the whole genetic picture. In order to circumvent this challenge, researchers can now use droplet digital PCR (ddPCR) on samples of plasma to screen for these same mutations, through a procedure colloquially termed â€œliquid biopsy.â€ The bodyâ€™s blood supply feeds primary tumors as well as metastases, and preliminary research suggests there is no selectivity in terms of which tumor cellsâ€™ DNA is deposited into the blood. In fact, normal and cancerous cells deposit into circulation, which makes the bloodstream an effective repository of cellular DNA, cancer and otherwise [9â€“11]. Thus, circulating blood contains traces of all variations of tumor DNA, which makes plasma tumor DNA (ptDNA) most representative of the varied genetic makeup of cancer cells. Sampling peripheral blood is also much easier than tissue biopsy, and the procedure comes with far fewer risks. There are many burgeoning applications of ddPCR within oncology, and breast cancer is poised to greatly benefit from the approach. The efficacy of ptDNA/ddPCR for detecting PIK3CA mutations in breast cancer has already been demonstrated with 93.3% sensitivity and 100% specificity in a study of 29 patients with early stage breast cancer [12]. It is important to clarify that this approach is specific for plasma, not serum. Research has shown the latter to not be reliable, whereas plasma has reproducible data supporting its efficacy [13, 14]. From identifying specific mutations sensitive to targeted therapy, to monitoring response to treatment, to detecting residual disease, this highly sensitive, cost effective approach offers improvement of breast cancer treatment and forward progress in studying the science behind it. This protocol describes the methodology for using ddPCR to detect tumor DNA in both plasma and primary tumor tissue. ")

 

Breast cancer is the leading cause of cancer in women and the second leading cause of cancer-related death. There are many subtypes of breast cancer, which can be identified through the process of molecular and genetic profiling. While the current standard of care utilizes tumor tissue biopsy to subclassify breast cancer, plasma tumor DNA (ptDNA) can be detected through droplet digital PCR (ddPCR) of plasma obtained from a simple blood draw. Tissue biopsy is not only more invasive but because tumors exhibit heterogeneity it can be less accurate. Blood collects DNA shed from normal and cancerous cells alike, thus ddPCR of plasma offers a broader picture of a cancer's genetic makeup. This chapter summarizes how patients with breast cancer can be screened for specific cancerous mutations in both tissue and plasma through the use of ddPCR. Every year, over 230,000 women are diagnosed with breast cancer, and nearly 40,000 die from the disease. It is the most commonly diagnosed cancer in wom

,journalId,title,abstract,keywords,clean_abstract,word_count
11,12,Detection of Cancer DNA in Early Stage and Met...,Breast cancer is the leading cause of cancer i...,Biopsy|Blood|Breast cancer|Droplet digital|Li...,Breast cancer is the leading cause of cancer i...,639
19,20,The Role of Telomerase in Breast Cancer's Resp...,"Currently, breast cancer appears to be the mos...",breast cancer|resistance|telomerase|therapy|ca...,"Currently, breast cancer appears to be the mos...",169
4,5,Cancer and cure: A critical analysis,Cancer is one of the most dreaded diseases of ...,cancer|cure.,Cancer is one of the most dreaded diseases of ...,452
5,6,Tumor microenvironment: recent advances in var...,This is a review regarding different types of ...,tumor|cancer.,This is a review regarding different types of ...,177
14,15,Field cancerization in breast cancer,Breast cancer affects one in seven women world...,cancer|breast cancer| cancer-adjacent tissues|...,Breast cancer affects one in seven women world...,254


In [11]:
import streamlit as st
import pickle
import requests

st.title('Sistem Rekomendasi Jurnal')
selected_journal_name = st.selectbox(
     'masukan judul jurnal',
     (["Measuring cancer evolution from the genome"]))

2024-08-21 15:30:41.180 
  command:

    streamlit run c:\Users\acer\anaconda3\envs\ki\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [12]:
def add_bg_from_url():
    st.markdown(
         f"""
         <style>
         .stApp {{
             background-image: url("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR60O2-muWvwgWvwmzklx-IydP5j0NZsLnHpg&usqp=CAU");
             background-attachment: fixed;
             background-size: cover
         }}
         </style>
         """,
         unsafe_allow_html=True
     )

In [13]:
add_bg_from_url()

In [14]:
#jurnal.loc[1]

In [15]:
#https://www.kaggle.com/code/saddamazyazy/klasifikasi-rekomendasi-jurnal